# Nonlinear Dynamics Model With an Extended Kalman Filter

## Imports

In [ ]:
import sys
sys.path.insert(0, "..")  # Add parent directory for module imports

import numpy as np
import matplotlib.pyplot as plt
import control as ctl
from kalman import ExtendedKalmanFilter
from lqr_controller import SwitchingLQRController
from simulation import TankDriveSimulation, discretize_system

## Parameters

In [ ]:
# Physical parameters
L = 0.5       # wheelbase [m]
tau = 0.2     # wheel speed time constant [s]

# State: [x, y, theta, v_l, v_r]
n_states = 5
n_inputs = 2
n_outputs = 4   # [theta, theta_dot, v_l, v_r]

# LQR weights
Q_lqr = np.diag([10.0, 10.0, 5.0, 0.1, 0.1])
R_lqr = np.diag([0.1, 0.1])

# Simulation parameters
T_final = 15.0
dt = 0.02   # 50 Hz update
N = int(T_final / dt)
t = np.arange(N) * dt

# Noise covariances
Q = np.diag([1e-5, 1e-5, 1e-6, 1e-4, 1e-4])  # process noise
R = np.diag([1e-3, 1e-2, 1e-3, 1e-3])  # measurement noise

# Control parameters (for LQR - computed from linearized model)
v0 = 0.5  # linearization point for LQR design

## Nonlinear Dynamics

### Continuous-Time Model

The **true** nonlinear dynamics are:

$$\dot{x} = \frac{v_l + v_r}{2} \cos(\theta)$$

$$\dot{y} = \frac{v_l + v_r}{2} \sin(\theta)$$

$$\dot{\theta} = \frac{v_r - v_l}{L}$$

$$\dot{v}_l = -\frac{1}{\tau}v_l + \frac{1}{\tau}u_l$$

$$\dot{v}_r = -\frac{1}{\tau}v_r + \frac{1}{\tau}u_r$$

### Discrete-Time Propagation

For the EKF, we integrate these ODEs using simple Euler integration (or a more sophisticated method):

$$\mathbf{x}_{k+1} = f(\mathbf{x}_k, \mathbf{u}_k)$$

In [ ]:
def nonlinear_dynamics(x, u, dt_step, L, tau):
    """
    Propagate state forward using nonlinear dynamics.
    
    Args:
        x: State [x, y, theta, v_l, v_r]
        u: Control input [u_l, u_r]
        dt_step: Time step
        L: Wheelbase [m]
        tau: Wheel speed time constant [s]
    
    Returns:
        x_next: Next state
    """
    x_pos, y_pos, theta, v_l, v_r = x
    u_l, u_r = u
    
    # Average velocity
    v_avg = (v_l + v_r) / 2.0
    
    # State derivatives
    x_dot = v_avg * np.cos(theta)
    y_dot = v_avg * np.sin(theta)
    theta_dot = (v_r - v_l) / L
    v_l_dot = (-v_l + u_l) / tau
    v_r_dot = (-v_r + u_r) / tau
    
    # Euler integration
    x_next = np.array([
        x_pos + x_dot * dt_step,
        y_pos + y_dot * dt_step,
        theta + theta_dot * dt_step,
        v_l + v_l_dot * dt_step,
        v_r + v_r_dot * dt_step
    ])
    
    return x_next

## Jacobian Computation

The EKF requires the **Jacobian** of the dynamics function with respect to the state:

$$F = \frac{\partial f}{\partial \mathbf{x}} \bigg|_{\mathbf{x} = \hat{\mathbf{x}}_k}$$

This is used to propagate the covariance:

$$P_{k+1|k} = F_k P_{k|k} F_k^T + Q$$

### Jacobian Matrix

$$F = \begin{bmatrix}
1 & 0 & -\frac{v_l + v_r}{2}\sin(\theta) \Delta t & \frac{1}{2}\cos(\theta) \Delta t & \frac{1}{2}\cos(\theta) \Delta t \\
0 & 1 & \frac{v_l + v_r}{2}\cos(\theta) \Delta t & \frac{1}{2}\sin(\theta) \Delta t & \frac{1}{2}\sin(\theta) \Delta t \\
0 & 0 & 1 & -\frac{\Delta t}{L} & \frac{\Delta t}{L} \\
0 & 0 & 0 & 1 - \frac{\Delta t}{\tau} & 0 \\
0 & 0 & 0 & 0 & 1 - \frac{\Delta t}{\tau}
\end{bmatrix}$$

In [ ]:
def compute_jacobian(x, dt_step, L, tau):
    """
    Compute Jacobian of dynamics function at state x.
    
    Args:
        x: State [x, y, theta, v_l, v_r]
        dt_step: Time step
        L: Wheelbase [m]
        tau: Wheel speed time constant [s]
    
    Returns:
        F: Jacobian matrix (5x5)
    """
    x_pos, y_pos, theta, v_l, v_r = x
    
    v_avg = (v_l + v_r) / 2.0
    
    F = np.eye(5)
    
    # df/dx
    F[0, 2] = -v_avg * np.sin(theta) * dt_step  # dx/dtheta
    F[0, 3] = 0.5 * np.cos(theta) * dt_step     # dx/dv_l
    F[0, 4] = 0.5 * np.cos(theta) * dt_step     # dx/dv_r
    
    # df/dy
    F[1, 2] = v_avg * np.cos(theta) * dt_step   # dy/dtheta
    F[1, 3] = 0.5 * np.sin(theta) * dt_step     # dy/dv_l
    F[1, 4] = 0.5 * np.sin(theta) * dt_step     # dy/dv_r
    
    # df/dtheta
    F[2, 3] = -dt_step / L                      # dtheta/dv_l
    F[2, 4] = dt_step / L                       # dtheta/dv_r
    
    # df/dv_l
    F[3, 3] = 1.0 - dt_step / tau               # dv_l/dv_l
    
    # df/dv_r
    F[4, 4] = 1.0 - dt_step / tau               # dv_r/dv_r
    
    return F

## Measurement Model

The measurement model is **linear** (same as before):

$$\mathbf{y} = C \mathbf{x} + \mathbf{v}$$

where $\mathbf{y} = [\theta, \dot{\theta}, v_l, v_r]^T$

In [ ]:
# IMU/encoder only: [theta, theta_dot, v_l, v_r]
C = np.array([
    [0, 0, 1,     0,    0],      # magnetometer: theta
    [0, 0, 0, -1/L, 1/L],        # gyro: theta_dot
    [0, 0, 0,     1,    0],      # left encoder: v_l
    [0, 0, 0,     0,    1],      # right encoder: v_r
])

def measurement_function(x):
    return C @ x

## LQR Controller Design

We still use LQR for control, which requires a linearized model. We'll use the same linearization as before around $v_0$.

## Extended Kalman Filter Implementation

### EKF Algorithm

**Predict Step** (uses nonlinear dynamics):
1. State prediction: $\hat{\mathbf{x}}_{k+1|k} = f(\hat{\mathbf{x}}_{k|k}, \mathbf{u}_k)$
2. Compute Jacobian: $F_k = \frac{\partial f}{\partial \mathbf{x}}\big|_{\hat{\mathbf{x}}_{k|k}}$
3. Covariance prediction: $P_{k+1|k} = F_k P_{k|k} F_k^T + Q$

**Update Step** (linear measurement model):
1. Innovation: $\mathbf{y}_k - h(\hat{\mathbf{x}}_{k|k})$ where $h(\mathbf{x}) = C\mathbf{x}$
2. Kalman gain: $K_k = P_{k|k-1} C^T (C P_{k|k-1} C^T + R)^{-1}$
3. State update: $\hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + K_k (\mathbf{y}_k - C\hat{\mathbf{x}}_{k|k-1})$
4. Covariance update: $P_{k|k} = (I - K_k C) P_{k|k-1}$

## Simulation Setup

In [ ]:
# Linearized model for LQR design
A = np.array([
    [0, 0,   0,      0.5,   0.5],
    [0, 0,   0.5*v0,  0.0,   0.0],
    [0, 0,   0,     -1/L,   1/L],
    [0, 0,   0,  -1.0/tau,  0.0],
    [0, 0,   0,      0.0, -1.0/tau],
])

B = np.array([
    [0.0,      0.0],
    [0.0,      0.0],
    [0.0,      0.0],
    [1.0/tau,  0.0],
    [0.0,  1.0/tau],
])

# Discretize for LQR and simulation
Ad, Bd, Cd, _ = discretize_system(A, B, C, np.zeros((n_outputs, n_inputs)), dt)

# Initial conditions
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
P_init = np.eye(n_states) * 0.1

# Initialize Extended Kalman Filter
ekf = ExtendedKalmanFilter(nonlinear_dynamics, compute_jacobian, C, Q, R, x0=x0, P0=P_init)

# Initialize LQR controller with reference switching
lqr = SwitchingLQRController(
    Ad, Bd, Q_lqr, R_lqr,
    x_ref_pos=np.array([5.0, 3.0, 0.0, v0, v0]),
    x_ref_stop=np.array([5.0, 3.0, 0.0, 0.0, 0.0]),
    target_pos=[5.0, 3.0],
    target_threshold=0.5,
    u_max=2.0
)

# Initialize simulation
system_matrices = {'Ad': Ad, 'Bd': Bd, 'Cd': Cd}
sim = TankDriveSimulation(
    system_matrices=system_matrices,
    filter_obj=ekf,
    controller_obj=lqr,
    Q=Q,
    R=R,
    dt=dt,
    N=N,
    random_seed=42
)

# Set initial conditions
sim.set_initial_conditions(x0_true=np.zeros(n_states), x0_hat=x0)

# Run simulation with nonlinear dynamics
results = sim.run(dynamics_func=nonlinear_dynamics, dynamics_params={'L': L, 'tau': tau})

print("EKF simulation complete!")

# Extract results
x_true = results['x_true']
x_hat_ekf = results['x_hat']
P_hist_ekf = results['P_hist']
t = results['t']

## Results Visualization

In [ ]:
# Compute position error and statistics
pos_error_ekf = sim.compute_position_error()
stats = sim.compute_statistics()

# Plot trajectory
plt.figure(figsize=(10, 6))
plt.plot(x_true[0, :], x_true[1, :], 'b-', linewidth=2, label='True path')
plt.plot(x_hat_ekf[0, :], x_hat_ekf[1, :], 'g--', linewidth=2, label='EKF estimate')
plt.plot(5.0, 3.0, 'rx', markersize=15, markeredgewidth=3, label='Target')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('EKF-Based Tank Drive Trajectory')
plt.show()

# Position error
plt.figure(figsize=(10, 4))
plt.plot(t, pos_error_ekf, 'b-', linewidth=2)
plt.xlabel('time [s]')
plt.ylabel('Position error [m]')
plt.grid(True, alpha=0.3)
plt.title('EKF Position Estimation Error')
plt.show()

# Position uncertainty
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(t, np.sqrt(P_hist_ekf[0, :]), 'b-', linewidth=2, label='σ_x')
plt.plot(t, np.sqrt(P_hist_ekf[1, :]), 'r-', linewidth=2, label='σ_y')
plt.ylabel('Position uncertainty [m]')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('EKF Position Uncertainty Growth')

plt.subplot(2, 1, 2)
plt.plot(t, np.sqrt(P_hist_ekf[2, :]), 'g-', linewidth=2, label='σ_θ')
plt.plot(t, np.sqrt(P_hist_ekf[3, :]), 'm-', linewidth=2, label='σ_v_l')
plt.plot(t, np.sqrt(P_hist_ekf[4, :]), 'c-', linewidth=2, label='σ_v_r')
plt.ylabel('Uncertainty [rad or m/s]')
plt.xlabel('time [s]')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Print statistics
print(f"\nFinal position uncertainty (1σ): x={stats['final_position_uncertainty']['x']:.4f} m, y={stats['final_position_uncertainty']['y']:.4f} m")
print(f"Mean position error: {stats['position_error']['mean']:.4f} m")
print(f"Max position error: {stats['position_error']['max']:.4f} m")